In [1]:
!pip install moviepy pydub git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-0nwqmp1z
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-0nwqmp1z
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)


In [3]:

import os
import whisper
from moviepy.editor import VideoFileClip
from pydub import AudioSegment

# Function to extract audio from video
def extract_audio(video_file, output_audio_file):
    print("Extracting audio from video...")
    video = VideoFileClip(video_file)
    audio = video.audio
    audio.write_audiofile(output_audio_file)
    print(f"Audio extracted to {output_audio_file}")

# Function to convert audio to WAV format mono PCM with 16kHz sample rate
def convert_audio(input_audio_file, output_audio_file):
    print("Converting audio to WAV format mono PCM with 16kHz sample rate...")
    audio = AudioSegment.from_file(input_audio_file)
    audio = audio.set_channels(1)  # Convert to mono
    audio = audio.set_frame_rate(16000)  # Set sample rate to 16kHz
    audio.export(output_audio_file, format="wav")
    print(f"Audio converted to {output_audio_file}")

# Function to transcribe audio using Whisper
def transcribe_audio_whisper(audio_file):
    model = whisper.load_model("base")
    result = model.transcribe(audio_file)
    return result['text']

# Function to add line breaks to transcript
def format_transcript(transcript, max_length=80):
    words = transcript.split()
    formatted_transcript = ""
    current_line = ""

    for word in words:
        if len(current_line) + len(word) + 1 > max_length:
            formatted_transcript += current_line + "\n"
            current_line = word
        else:
            if current_line:
                current_line += " " + word
            else:
                current_line = word

    if current_line:
        formatted_transcript += current_line

    return formatted_transcript

# Main function to generate transcript
def generate_transcript(video_file):
    temp_audio_file = "temp_audio.mp3"
    final_audio_file = "final_audio.wav"

    # Extract audio from video
    extract_audio(video_file, temp_audio_file)
    # Convert audio to WAV format mono PCM with 16kHz sample rate
    convert_audio(temp_audio_file, final_audio_file)

    # Transcribe the entire audio file using Whisper
    print("Transcribing audio file...")
    transcript = transcribe_audio_whisper(final_audio_file)

    # Clean up temporary audio files
    os.remove(temp_audio_file)
    os.remove(final_audio_file)
    print("Temporary files cleaned up.")

    # Format transcript with line breaks
    formatted_transcript = format_transcript(transcript)

    return formatted_transcript

# Example usage in Jupyter Notebook or Google Colab
video_file = "/content/260 - Sorting Strings.mp4"  # Replace with your video file path

print("Generating transcript...")
transcript = generate_transcript(video_file)
print("Transcript generated:")
print(transcript)

# Optionally save the transcript to a file
with open("transcript.txt", "w") as f:
    f.write(transcript)
print("Transcript saved to transcript.txt")


Generating transcript...
Extracting audio from video...
MoviePy - Writing audio in temp_audio.mp3


MoviePy - Done.
Audio extracted to temp_audio.mp3
Converting audio to WAV format mono PCM with 16kHz sample rate...
Audio converted to final_audio.wav
Transcribing audio file...


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Temporary files cleaned up.
Transcript generated:
The next thing we're going to take a look at is how we can sort in array of
strings. So I'm going to update my data array. I'm going to change it to T,
capital A, lowercase A, capital B and lowercase B. Just a little bit ago before
we introduced this comparator function, I told you that by default JavaScript is
going to take all the elements inside an array, turn them into strings, and then
compare them. So that might lead you to believe that JavaScript can sort in
array of strings really well. Once again, not quite the case. So let me
demonstrate that to you very quickly. I'm going to remove the comparator
function because this one is really only designed to work with numbers. So I'm
going to try calling data.sort and I'm going to see what I get back. So I'm
going to run that and I get back something that definitely looks like it has
changed, but it's not quite what I think I would be looking for if I was sorting
in array of strings. I

In [4]:
!pip install transformers torch


In [5]:
import os
from transformers import BartForConditionalGeneration, BartTokenizer

class BartSummarizer:
    def __init__(self, model_name='facebook/bart-large-cnn'):
        self.model_name = model_name
        self.tokenizer = BartTokenizer.from_pretrained(model_name)
        self.model = BartForConditionalGeneration.from_pretrained(model_name)

    def summarize_text(self, text, min_length=50, max_length=200):
        inputs = self.tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = self.model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

    def summarize_file(self, file_path, min_length=50, max_length=200):
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        with open(file_path, 'r') as file:
            text = file.read()
        return self.summarize_text(text, min_length, max_length)

    def customize_summarization(self, input_content, min_length=50, max_length=200, input_type='text'):
        if input_type == 'text':
            return self.summarize_text(input_content, min_length, max_length)
        elif input_type == 'file':
            return self.summarize_file(input_content, min_length, max_length)
        else:
            raise ValueError("Invalid input_type. Use 'text' or 'file'.")

# Example usage:
if __name__ == "__main__":
    summarizer = BartSummarizer()

    # Example: Summarize text
    text = "The quick brown fox jumps over the lazy dog. " * 100
    summary = summarizer.customize_summarization(text, min_length=50, max_length=200, input_type='text')
    print("Summary from text input:")
    print(summary)

    # Example: Summarize text from a file
    file_path = "/content/transcript.txt"  # Make sure this file exists
    summary = summarizer.customize_summarization(file_path, min_length=50, max_length=200, input_type='file')
    print("\nSummary from file input:")
    print(summary)


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Summary from text input:
The quick brown fox jumps over the lazy dog. The quick brownFox.com video shows how the foxes react to the dog's behavior. The foxes jump over the dog and then the dog jumps over them. The dog then jumps back over the fox and the fox then jumps over it.

Summary from file input:
By default, JavaScript is going to take all the elements inside an array, turn them into strings, and thencompare them. We need to put in a slightly different implementation to tell sort how to work with strings. This is a built in function that belongs to all strings in JavaScript.
